In [2]:
import numpy as np
from astropy.io import ascii
from pylab import *
from astropy.table import Table, vstack
import pandas as pd
from expecto import get_spectrum
from scipy.interpolate import RegularGridInterpolator as rgi
from scipy.stats import binned_statistic as bin
from itertools import product
import batman
#%run Interpolate.ipynb
#%run GetSpectra.ipynb
#%run Binning.ipynb

In [1]:
'''
tweaked binning function that allows binning at every point in the pwv timeseries
'''

def res_bin_timestamp(flux, wavelength, res, wave_in, wave_fin, spectrum_type="stellar"):
    #print(wavelength)
    #print
    #print(flux)
    wave_in = wavelength[0]
    wave_fin = wavelength[-1]
    wave_array = [wave_in]
    n=wave_array[0]
    i=0
    #print(wave_in)
    #print(wave_fin)
    while wave_in <= n <= wave_fin:
        if wave_array[i] >= wave_fin:
            break
        bin_size = wave_array[i]/res
        if wave_array[i]+bin_size >= wave_fin:
            break
        wave_array.append(wave_array[i]+bin_size)
        n=wave_array[i]
        i+=1
    #print(wave_array)
    
    
        
    if spectrum_type == "stellar":
        
        wave_in_index = 0
        wave_out_index = -1
        
        '''
        use this section if using binning function individually and need to specify wave range
        wave_in_index = np.where(wavelength == wave_in*10)[0]
        print(wave_in_index)
        wave_out_index = np.where(wavelength == wave_fin*10)[0]
        print(wave_out_index)
        '''

        wavelength_range = wavelength[wave_in_index:wave_out_index]
        #print(wavelength_range)
        #finds step size of stellar spectrum
        
        high_res_step = [wavelength_range[1]-wavelength_range[0]]
        j=0
        for n in wavelength:
            if j == len(wavelength_range)-1:
                break
            bin_size2 = wavelength_range[j+1] - wavelength_range[j]
            high_res_step.append(bin_size2)
            j+=1
        
        #flux must be multiplied by fractional step size in order to scale it for binning
        #flux = (spectrum.flux.value)*high_res_step
        flux_new = (flux[wave_in_index:wave_out_index])
        #print(high_res_step)
        #print(len(flux_new))
        #print(len(wavelength_range))
        #print(len(flux_new))
        
        
    elif spectrum_type == "earth":
        wavelength = spectrum["col1"]
        flux = spectrum["col2"]
    
    #plt.hist(wavelength_range, bins=wave_array, weights=flux_new)
    #plt.show()
    #return np.histogram(wavelength_range, bins=wave_array, weights=flux_new)
    return bin(wavelength_range, flux_new, bins=wave_array, statistic="mean")


In [ ]:
#spectrum = full_grid[0]

def res_bin_general(flux, wavelength, res, wave_in, wave_fin, spectrum_type="stellar"):
    #print(wavelength)
    wave_array = [wave_in]
    n=wave_array[0]
    i=0
    while wave_in <= n <= wave_fin:
        bin_size = wave_array[i]/res
        if wave_array[i]+bin_size >= wave_fin:
            break
        wave_array.append(wave_array[i]+bin_size)
        n=wave_array[i]
        i+=1
        
    if spectrum_type == "stellar":
        wave_in_index = np.where(wavelength.value == wave_in)[0]
        #print(wave_in_index)
        wave_out_index = np.where(wavelength.value == wave_fin)[0]
        #print(wave_out_index)

        wavelength_range = wavelength[wave_in_index[0]:wave_out_index[0]].value
        #print(wavelength_range)
        #finds step size of stellar spectrum
        
        high_res_step = [wavelength_range[0]-wavelength_range[1]]
        j=0
        for n in wavelength:
            if j == len(wavelength_range)-1:
                break
            bin_size2 = wavelength_range[j+1] - wavelength_range[j]
            high_res_step.append(bin_size2)
            j+=1
        
        #flux must be multiplied by fractional step size in order to scale it for binning
        #flux = (spectrum.flux.value)*high_res_step
        #print(flux[wave_in_index[0]:wave_out_index[0]])
        #print(high_res_step)
        flux_new = (flux[wave_in_index[0]:wave_out_index[0]])*high_res_step
        
        
        #print(len(flux_new))
        #print(len(wavelength_range))
        #print(wave_array)
        
    elif spectrum_type == "earth":
        wavelength = spectrum["col1"]
        flux = spectrum["col2"]
    
    plt.hist(wavelength_range, bins=wave_array, weights=flux_new)
    plt.show()
    return np.histogram(wavelength_range, bins=wave_array, weights=flux_new)


In [1]:
'''
function that takes in stellar spectrum, wave range, and res. Output is timeseries of flux based on pwv variability,
one timeseries for each wavelength. 
'''


def binned_flux_pwv_v2(spectrum, pwv_input, wave_in, wave_out, res):
    stellar = spec_wave_range(spectrum, wave_in, wave_out)
    stellar_wave = stellar[0]
    stellar_flux = stellar[1]
    #pwv = np.load("pwv_target.npy")
    
    
    transmission_array = []
    
    i=0
    for n in stellar_wave:
        str_value = str(stellar_wave[i])
        value = stellar_wave[i]/10
        stellar_flux_value = stellar_flux[i]
        transmission  = interpolate_water(pwv_input, value)
        flux = stellar_flux_value*transmission
        d[str_value+"_trans_array"] = np.array(flux)
    
        transmission_array.append(d[str_value+"_trans_array"])
        i+=1
        
    transmission_array = np.array(transmission_array)
    
    time_array = np.arange(0, len(pwv_input), 1)
    
    transmission_array_binned = []
    #wave_in_converted = wave_in*1000
    #wave_out_converted = wave_out*1000
    wave_in_converted = wave_in*100
    wave_out_converted = wave_out*100
    
    #print(stellar_wave)
    j=0
    #wave_array = res_bin_timestamp(transmission_array[:,j], stellar_wave, res, wave_in_converted, wave_out_converted)[1]
    for n in time_array:
        
        bin_1 = res_bin_timestamp(transmission_array[:,j], stellar_wave, res, wave_in_converted, wave_out_converted)
        transmission_array_binned.append(bin_1)
        j+=1
    
    return transmission_array_binned

In [3]:
'''
function that takes in stellar spectrum, wave range, and res. Output is timeseries of flux based on pwv variability,
one timeseries for each wavelength. 

This version adds in exoplanet transit.
'''

'''
def binned_flux_pwv_transit(spectrum, pwv_input, wave_in, wave_out, res, rp = .07):
    stellar = spec_wave_range(spectrum, wave_in, wave_out)
    stellar_wave = stellar[0]
    stellar_flux = stellar[1]
    #pwv = np.load("pwv_target.npy")
    t = np.linspace(-0.025, 0.025, len(pwv_input))
    bulk_transit_depth = rp**2

    
    transmission_array = []
    
    i=0
    for n in stellar_wave:
        str_value = str(stellar_wave[i])

        value = stellar_wave[i]/10
        stellar_flux_value = stellar_flux[i]
        transmission  = interpolate_water(pwv_input, value)
        
        #Satm_transit_i = line that gets transit depth of atm in ppm at wavelength
        
        total_transit_depth = bulk_transit_depth + atm_transit_i
        rp_i = np.sqrt(total_transit_depth)
        params.rp = rp_i
        model = batman.TransitModel(params, t)
        transit = model.light_curve(params)
        
        flux = stellar_flux_value*transmission
        transit_flux = tranist*flux
        
        d[str_value+"_trans_array"] = np.array(transit_flux)
        
        transmission_array.append(d[str_value+"_trans_array"])
        i+=1
        
    transmission_array = np.array(transmission_array)
    
    time_array = np.arange(0, 627, 1)
    
    transmission_array_binned = []
    wave_in_converted = wave_in*1000
    wave_out_converted = wave_out*1000
    
    j=0
    for n in time_array:
        bin_1 = res_bin_timestamp(transmission_array[:,j], stellar_wave, res, wave_in_converted, wave_out_converted)
        transmission_array_binned.append(bin_1)
        j+=1
    
    return(transmission_array_binned)
'''

In [ ]:
'''
function that takes in stellar spectrum, wave range, and res. Output is timeseries of flux based on pwv variability,
one timeseries for each wavelength. This version isn't binned, to test whether residual pwv pattern is a binning problem
'''


def flux_pwv(spectrum, wave_in, wave_out, res):
    stellar = spec_wave_range(spectrum, wave_in, wave_out)
    stellar_wave = stellar[0]
    stellar_flux = stellar[1]
    pwv = np.load("pwv_target.npy")
    
    transmission_array = []
    
    i=0
    for n in stellar_wave:
        str_value = str(stellar_wave[i])
        value = stellar_wave[i]/10
        stellar_flux_value = stellar_flux[i]
        transmission  = interpolate_water(pwv, value)
        flux = stellar_flux_value*transmission
        d[str_value+"_trans_array"] = np.array(flux)
    
        transmission_array.append(d[str_value+"_trans_array"])
        i+=1
        
    transmission_array = np.array(transmission_array)
    '''
    time_array = np.arange(0, 627, 1)
    
    transmission_array_binned = []
    wave_in_converted = wave_in*1000
    wave_out_converted = wave_out*1000
    
    j=0
    for n in time_array:
        bin_1 = res_bin_timestamp(transmission_array[:,j], stellar_wave, res, wave_in_converted, wave_out_converted)
        transmission_array_binned.append(bin_1)
        j+=1
    '''
    return(transmission_array)

In [ ]:
'''
function that takes in stellar spectrum, wave range, and res. Output is timeseries of flux based on pwv variability,
one timeseries for each wavelength. 
'''


def binned_flux_pwv_v2(spectrum, pwv_input, wave_in, wave_out, res):
    stellar = spec_wave_range(spectrum, wave_in, wave_out)
    stellar_wave = stellar[0]
    stellar_flux = stellar[1]
    #pwv = np.load("pwv_target.npy")
    
    
    transmission_array = []
    
    i=0
    for n in stellar_wave:
        str_value = str(stellar_wave[i])
        value = stellar_wave[i]/10
        stellar_flux_value = stellar_flux[i]
        transmission  = interpolate_water(pwv_input, value)
        flux = stellar_flux_value*transmission
        d[str_value+"_trans_array"] = np.array(flux)
    
        transmission_array.append(d[str_value+"_trans_array"])
        i+=1
        
    transmission_array = np.array(transmission_array)
    
    time_array = np.arange(0, len(pwv_input), 1)
    
    transmission_array_binned = []
    wave_in_converted = wave_in*1000
    wave_out_converted = wave_out*1000
    
    j=0
    for n in time_array:
        bin_1 = res_bin_timestamp(transmission_array[:,j], stellar_wave, res, wave_in_converted, wave_out_converted)
        transmission_array_binned.append(bin_1)
        j+=1
    
    return(transmission_array_binned)

In [3]:
def bin_array(wave_in, wave_out, res):
    wave_array = [wave_in]
    n=wave_array[0]
    i=0
    while wave_in <= n <= wave_out:
        bin_size = wave_array[i]/res
        if wave_array[i]+bin_size >= wave_out:
            break
        wave_array.append(wave_array[i]+bin_size)
        n=wave_array[i]
        i+=1
    return wave_array